In [1]:
import time, os, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, random_split
import numpy as np
from google.colab import drive
from torch.utils.data import DataLoader, Subset
import shutil

In [2]:
!rm -rf /content/Hani-Raft-Repo
!rm -rf FoV_ML_spatial_audio_prediction
!git clone --recurse-submodules https://github.com/AI01010/FoV_ML_spatial_audio_prediction.git

os.chdir("FoV_ML_spatial_audio_prediction/Scripts/finalCode")
!git pull origin main

Cloning into 'FoV_ML_spatial_audio_prediction'...
remote: Enumerating objects: 1377, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 1377 (delta 171), reused 136 (delta 78), pack-reused 1111 (from 2)
Receiving objects: 100% (1377/1377), 1.54 GiB | 42.78 MiB/s, done.
Resolving deltas: 100% (611/611), done.
Updating files: 100% (615/615), done.
Filtering content: 100% (4/4), 70.06 MiB | 48.61 MiB/s, done.
Submodule 'Hani-Raft-Repo' (https://github.com/HaniKamran/RAFT) registered for path 'Hani-Raft-Repo'
Submodule 'U-2-Net-Repo' (https://github.com/xuebinqin/U-2-Net) registered for path 'U-2-Net-Repo'
Cloning into '/content/FoV_ML_spatial_audio_prediction/Hani-Raft-Repo'...
remote: Enumerating objects: 179, done.        
remote: Counting objects: 100% (31/31), done.        
remote: Compressing objects: 100% (31/31), done.        
remote: Total 179 (delta 18), reused 0 (delta 0), pack-reused 148 (from 3)        
Receiv

In [ ]:
print(os.getcwd())
from cnn_model import HeatmapFusionCNN, SaliencyTileDataset
# Clear GPU memory
torch.cuda.empty_cache() if torch.cuda.is_available() else None

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB\n")

# Configuration - 25% of 4K resolution
FRAME_WIDTH = 1920    # 25% of 3840
FRAME_HEIGHT = 960   # 25% of 1920
TILES_X = 16
TILES_Y = 9
NUM_TILES = TILES_X * TILES_Y  # 144 tiles
TILE_WIDTH = FRAME_WIDTH // TILES_X   # 60 pixels
TILE_HEIGHT = FRAME_HEIGHT // TILES_Y  # ~53 pixels
NUM_HEATMAPS = 9  # 7 audio + 2 video

# Video configuration
FPS = 60  # Frames per second (full framerate)
FRAME_SAMPLE_RATE = 5  # Take every 5th frame

print(f"Configuration:")
print(f"  Frame size: {FRAME_WIDTH}x{FRAME_HEIGHT} (25% of 4K)")
print(f"  Grid: {TILES_X}x{TILES_Y} = {NUM_TILES} tiles")
print(f"  Tile size: {TILE_WIDTH}x{TILE_HEIGHT}")
print(f"  Heatmaps per frame: {NUM_HEATMAPS}")


/content/FoV_ML_spatial_audio_prediction/Scripts/finalCode
Using device: cuda
GPU: Tesla T4
Memory: 15.83 GB

Configuration:
  Frame size: 1920x960 (25% of 4K)
  Grid: 16x9 = 144 tiles
  Tile size: 120x106
  Heatmaps per frame: 9


In [4]:
drive.mount('/content/drive')
data_dir = "/content/drive/MyDrive/InputFiles/"

x = []
y = []

videos = ["0001", "0002", "0003"]
frames = ["0", "1000", "2000"]

for vid in videos:
  for frame in frames:
    xFile = np.load(data_dir + vid + "_" + frame + "_heatmaps.npy", mmap_mode="r")
    yFile = np.load(data_dir + vid + "_" + frame + "_labels.npy", mmap_mode="r")
    x.append(xFile)
    y.append(yFile)

dataset = SaliencyTileDataset(x, y)

# Total length
total_len = len(dataset)
train_size = int(0.9 * total_len)
val_size = total_len - train_size

# Shuffled indices (shuffle only once)
shuffledIndices = torch.randperm(total_len)

# Create subsets
train_dataset = Subset(dataset, shuffledIndices[0:train_size])
val_dataset = Subset(dataset, shuffledIndices[train_size:])

# Dataloaders
trainLoader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=6, pin_memory=True)
valLoader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
model = HeatmapFusionCNN(num_heatmaps=NUM_HEATMAPS, numCols = TILES_X, numRows = TILES_Y, theDevice = device).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {total_params:,}\n")

model.trainModel(trainLoader, valLoader)

torch.save(model.state_dict(), "cnn_model.pth")

save_dir = '/content/drive/MyDrive/FinalModel'
os.makedirs(save_dir, exist_ok=True)
drive_file = os.path.join(save_dir, f"cnn_model.pth")
local_file = f"./cnn_model.pth"

shutil.copy(local_file, drive_file)

if os.path.exists(drive_file):
    print(f"✅ File exists in Drive: {drive_file}")
else:
    print(f"❌ File not found: {drive_file}")

drive.flush_and_unmount()  # flush any pending writes
drive.mount('/content/drive')

Streaming output truncated to the last 5000 lines.
After pool: 0.0 4.363923072814941
After fc1: -2.835864305496216 9.182629585266113
After fc2 (output): -9.577896118164062 13.734170913696289
Outputs are these:  -9.577896118164062 13.734170913696289
Batch 24/51: 0.7s
before fusion: 0.0 1.0
After fusion: -0.9424377083778381 1.1126811504364014
After conv1: -0.8299866318702698 0.8723360896110535
After conv2: -3.008265972137451 3.002805709838867
After conv3: -2.527723789215088 2.8737363815307617
After pool: 0.0 4.640234470367432
After fc1: -2.211599826812744 8.710116386413574
After fc2 (output): -7.839722156524658 12.77249813079834
Outputs are these:  -7.839722156524658 12.77249813079834
Batch 25/51: 0.7s
before fusion: 0.0 1.0
After fusion: -0.956443190574646 1.1365575790405273
After conv1: -0.9146043658256531 0.8667693734169006
After conv2: -3.213043689727783 3.680954933166504
After conv3: -2.412203550338745 3.0321028232574463
After pool: 0.0 4.2112040519714355
After fc1: -2.2245841026306